In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

A primeira coisa que vamos fazer é ler o conjunto de dados usando a função read_csv() dos Pandas.
Colocaremos esses dados em um DataFrame do Pandas, chamado "titanic", e nomearemos cada uma das colunas.

In [ ]:
url = 'https://raw.githubusercontent.com/BigDataGal/Python-for-Data-Science/master/titanic-train.csv'
titanic = pd.read_csv(url)
titanic.columns = ['PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
titanic.head()

**VARIABLE DESCRIPTION**
 * **Survived** - Survival (0 = No; 1 = Yes);
 * **Pclass** - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd);
 * **Name** - Name;
 * **Sex** - Sex;
 * **Age** - Age;
 * **SibSp** - Number of Siblings/Spouses Aboard;
 * **Parch** - Number of Parents/Children Aboard;
 * **Ticket** - Ticket Number;
 * **Fare** - Passenger Fare (British pound);
 * **Cabin** - Cabin;
 * **Embarked** - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton);

Como estamos construindo um modelo para prever a sobrevivência de passageiros do Titanic, nosso alvo será a variável "Survived" do dataframe titanic.
Para ter certeza de que é uma variável binária, vamos usar a função countplot () do Seaborn.


In [ ]:
sb.countplot(x='Survived',data=titanic, palette='hls')

###Checking for missing values
É fácil checar missing values usando método isnull() com o método sum(), o número retornado condiz com a quantidade True para o teste, ou seja, quantidade de valores nulos nas variáveis

In [ ]:
titanic.isnull().sum()

In [ ]:
titanic.info()

Ok, então existem 891 linhas no dataframe. Cabin é quase todo composto por missing values, então podemos eliminar essa variável completamente! Mas e quanto à idade? A age parece um preditor relevante para a sobrevivência, certo? Nós queremos manter as variáveis, mas tem 177 missing values. Precisamos encontrar uma maneira de nos aproximarmos desses valores em falta!

## Lidando com missing values
#### Removendo missing values
Vamos além dos missing values... Vamos descartar todas as variáveis que não são relevantes para a predição de Survived.

In [ ]:
titanic.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)


In [ ]:
titanic.head()

In [ ]:
titanic.isnull().sum()

Agora, o dataframe foi reduzido para apenas variáveis relevantes, mas agora precisamos lidar com os valores ausentes na variável age.

## Imputing missing values
Vejamos como a idade do passageiro está relacionada à sua classe como passageiro no barco.

In [ ]:
sb.boxplot(x='Pclass', y='Age', data=titanic, palette='hls')

Falando mais ou menos, poderíamos dizer que quanto mais jovem é um passageiro, mais provável é que ele esteja na 3ª classe. Quanto mais velho for um passageiro, maior a probabilidade de estarem na 1ª classe. Portanto, há um relacionamento frouxo entre essas variáveis. Então, vamos escrever uma função que se aproxime da idade dos passageiros, com base em sua classe. Na caixa, parece que a idade média dos passageiros de 1ª classe é de cerca de 37 anos, os passageiros de 2ª classe são 29 e os passageiros de 3ª classe são 24.

Então, vamos escrever uma função que encontre cada valor nulo na variável Age e, para cada nulo, verifique o valor da Pclass e atribua um valor de idade de acordo com a idade média dos passageiros nessa classe.

In [ ]:
def age_approx(cols):
    Age = cols[0]
    Pclass = cols[1]

    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        else:
            return 24
    else:
        return Age


Após definir a função, vamos executar apenas para os valores nulos de Age

In [ ]:
titanic[['Age', 'Pclass']]

In [ ]:
titanic[['Age', 'Pclass']].apply(age_approx, axis=1)

In [ ]:
titanic['Age'] = titanic[['Age', 'Pclass']].apply(age_approx, axis=1)
titanic.isnull().sum()

Existem ainda 2 valores nulos na variável Embarked. Podemos eliminar esses dois registros sem perder muitas informações importantes do nosso conjunto de dados, então faremos isso.

In [ ]:
titanic.dropna(inplace=True)
titanic.isnull().sum()

In [ ]:
titanic.info()

### Convertendo variaveis categóricas com one-hot-enconding (via pandas usa-se a função get_dummies)

In [ ]:
aux = pd.get_dummies(titanic['Sex'],drop_first=False)
aux.head()

In [ ]:
gender = pd.get_dummies(titanic['Sex'],drop_first=True)
gender.head()

In [ ]:
embark_location = pd.get_dummies(titanic['Embarked'],drop_first=True)
embark_location.head(10)

In [ ]:
titanic.drop(['Sex', 'Embarked'],axis=1,inplace=True)
titanic.head()

titanic_dmy = pd.concat([titanic,gender,embark_location],axis=1)
titanic_dmy.head()

In [ ]:
titanic_dmy.info()

Precisariamos fazer Feature Selection, mas faremos nas próximas aulas...


### Agora Vamos Classificar!
1º: Separar o conjunto em variável resposta e variáveis de treinamento

In [ ]:
X = titanic_dmy.iloc[:,[1,2,3,4,5,6,7,8]].values
y = titanic_dmy.iloc[:,0].values

In [ ]:
#X = titanic_dmy[['Age',	'SibSp',	'Parch',	'Fare',	'male'	,'S']].values
#y = titanic_dmy['Survived'].values

In [ ]:
X

In [ ]:
y

Agora dividir em treino e teste (teste com 30%)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=25)

Normalização do Conjunto

**Pergunta**: Devemos rodar o Scaler em qual conjunto de dados (completo, train ou teste)?

In [ ]:
#scaler = MinMaxScaler().fit(X_train)
#scaler = MinMaxScaler().fit(X_test)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

In [ ]:
X_test

Criação do modelo knn com o valor de k =




In [ ]:
Classif_KNN = KNeighborsClassifier(n_neighbors=1)

Processo de treinamento com 70% dos dados

In [ ]:
Classif_KNN.fit(X_train, y_train)

Processo de teste com 30% dos dados que não foram utilizados no treinamento

In [ ]:
y_pred = Classif_KNN.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_test

###Avaliando o modelo
Analisando o rotulo real com o rotulo predito

In [ ]:
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
cm = confusion_matrix( y_test, y_pred)
disp = ConfusionMatrixDisplay(cm)
disp.plot()

In [ ]:
labels = [0,1]
label_names = ['NS', 'S']
cm = confusion_matrix( y_test, y_pred, labels = labels, normalize='true')
disp = ConfusionMatrixDisplay(cm, display_labels = label_names)
disp.plot()

Testa do K=1 até K=26


In [ ]:
k_range = range(1, 26)
scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

In [ ]:
print(scores)
#Plota os valores de acc. em função do valor escolhido de K
plt.plot(k_range, scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Testing Accuracy')

In [ ]:
max(scores)

In [ ]:
scores.index(max(scores))

In [ ]:
#Avaliando a melhor configuração de KNN
best_KNN = KNeighborsClassifier(n_neighbors=10)
best_KNN.fit(X_train, y_train)
y_pred = best_KNN.predict(X_test)


In [ ]:
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
cm = confusion_matrix( y_test, y_pred, labels = labels, normalize='true')
disp = ConfusionMatrixDisplay(cm, display_labels = label_names)
disp.plot()

## Exercício:

Fizemos esta demonstração com o MinMaxScaler. Reproduza os estudos para os seguintes cenários: Sem Data Scaling, StandardScaler e RobustScaler

Analise os resultados e responda as seguintes perguntas:

*   Qual é o efeito do normalizador/padronizador sobre o parâmetro do classificador?
*   Qual é o efeito na matriz de confusão?
*   Qual é o efeito na acurácia?
*   Qual é o melhor na sua opinião?


## Desafio (opcional):

No estudo de varreduras do parâmetro k do `KNeighborsClassifier`, notamos algumas flutuações na acurácia do modelo. Faça uma alteração no programa para que seja possível melhorar a avaliação do melhor valor de k.
